To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


# ===============================================
# 1. 구글 드라이브 연동 및 경로 설정
# ===============================================

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import os
# 모델과 결과물을 저장할 기본 경로 설정
# 본인의 구글 드라이브에 'llama3-8b-sft' 라는 폴더가 생성됩니다.
DRIVE_PATH = "/gdrive/MyDrive/llama3-8b-sft-r1-cot"
if not os.path.exists(DRIVE_PATH):
    os.makedirs(DRIVE_PATH)

In [4]:
# 학습 결과(체크포인트)가 저장될 경로
OUTPUT_DIR = os.path.join(DRIVE_PATH, "outputs")
# 최종 병합된 모델이 저장될 경로
MERGED_MODEL_PATH = os.path.join(DRIVE_PATH, "llama3-8b-sft-merged")

In [ ]:
print(f"구글 드라이브 마운트 완료!")
print(f"학습 결과 저장 경로: {OUTPUT_DIR}")
print(f"병합 모델 저장 경로: {MERGED_MODEL_PATH}")

### Installation

In [5]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

## Wandb 설치

In [6]:
%%capture
!pip install wandb

## Wandb 로그인

In [7]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: youngjae-you (hansungit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Unsloth

In [8]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit", # 순수 SFT에 유리: base 모델은 사전 학습된 상태로, 특정 태스크에 대한 명시적인 지시(instruction)를 따르도록 훈련되지 않았습니다. 따라서 sequelbox/Celestia3-DeepSeek-R1-0528 데이터를 사용하여 새로운 추론(Reasoning) 능력을 주입하고 싶을 때, base 모델은 해당 데이터의 분포와 지시에 더 순수하게 적응할 수 있습니다. 이미 instruct 모델은 대화 형식이나 특정 지시를 따르도록 추가로 학습되었기 때문에, DeepSeek-R1 데이터의 형식이 instruct 모델의 기존 학습 방식과 충돌할 수 있습니다.
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.7.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [10]:
from datasets import load_dataset

# 2. DeepSeek-R1 생성 데이터셋 로드 및 포맷팅
# Hugging Face Hub에서 데이터셋을 로드합니다.
print("데이터셋을 로드합니다...")
distillation_dataset = load_dataset("sequelbox/Celestia3-DeepSeek-R1-0528", split="train")
print("데이터셋 로드 완료.")

데이터셋을 로드합니다...


README.md: 0.00B [00:00, ?B/s]

Celestia3_DSR10528_Part1.csv:   0%|          | 0.00/208M [00:00<?, ?B/s]

Celestia3_DSR10528_Part2.csv:   0%|          | 0.00/223M [00:00<?, ?B/s]

Celestia3_DSR10528_Part3.csv:   0%|          | 0.00/226M [00:00<?, ?B/s]

Celestia3_DSR10528_Part4.csv:   0%|          | 0.00/209M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90990 [00:00<?, ? examples/s]

데이터셋 로드 완료.


In [11]:
distillation_dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 90990
})

In [12]:
from unsloth.chat_templates import get_chat_template

# Llama 3.1에 맞는 채팅 템플릿을 가져옵니다.
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1", # Llama 3.1 템플릿 사용
)

In [13]:
# # 데이터셋을 Llama 3.1 채팅 템플릿에 맞는 'conversations' 형식으로 포맷팅
# # 'prompt' 컬럼은 'user' 메시지로, 'completion' 컬럼은 'assistant' 메시지로 매핑합니다.
# def format_for_llama31_template(example):
#     # Llama 3.1의 기본 채팅 템플릿은 시스템 메시지를 지원하지만,
#     # 이 데이터셋은 'prompt'와 'completion'만 있으므로,
#     # 간단한 사용자-어시스턴트 대화 형식으로 구성합니다.
#     # 만약 시스템 메시지를 추가하고 싶다면, 여기에 하드코딩하거나 데이터셋에 추가해야 합니다.
#     messages = [
#         {"role": "user", "content": example["prompt"]},
#         {"role": "assistant", "content": example["completion"]},
#     ]
#     return {"conversations": messages}

# 데이터셋을 Llama 3.1 채팅 템플릿에 맞는 'conversations' 형식으로 포맷팅
# 'prompt' 컬럼은 'user' 메시지로, 'completion' 컬럼은 'assistant' 메시지로 매핑합니다.
def format_for_llama31_template_with_tokenizer(examples):
    # 'examples'는 { 'prompt': [...], 'completion': [...] } 형태의 딕셔너리입니다.
    # 각 샘플에 대한 'text' (포맷팅된 문자열) 리스트를 담을 최종 리스트를 생성합니다.
    all_formatted_texts = []

    # 배치 내의 각 샘플을 순회합니다.
    for prompt, completion in zip(examples["prompt"], examples["completion"]):
        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": completion},
        ]
        # tokenizer.apply_chat_template을 사용하여 Llama 3.1 템플릿에 맞춰 포맷팅합니다.
        # tokenize=False: 아직 토큰화하지 않고 문자열 형태로 반환합니다. (SFTTrainer가 나중에 토큰화)
        # add_generation_prompt=False: 어시스턴트의 답변이 이미 있으므로 생성 프롬프트를 추가하지 않습니다.
        formatted_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        all_formatted_texts.append(formatted_text)

    # 반환할 때는 'text' 컬럼에 대한 리스트를 딕셔너리 형태로 반환합니다.
    return {"text": all_formatted_texts}

distillation_formatted_dataset = distillation_dataset.map(
    format_for_llama31_template_with_tokenizer,
    remove_columns=["prompt", "completion"], # 원본 컬럼 제거
    batched=True, # 배치 처리 활성화 (권장)
    desc="Formatting dataset for Llama 3.1 chat template using tokenizer"
)

Formatting dataset for Llama 3.1 chat template using tokenizer:   0%|          | 0/90990 [00:00<?, ? examples/…

In [14]:
# 데이터셋의 첫 번째 예시 확인 (선택 사항)
print("\n포맷팅된 데이터셋의 첫 번째 예시:")
print(distillation_formatted_dataset[0])


포맷팅된 데이터셋의 첫 번째 예시:
{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCan you provide an example of a problem where a greedy algorithm is not optimal, but a dynamic programming algorithm is?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<think>\r\n\r\nWe are looking for a problem where a greedy approach fails to produce the optimal solution, but dynamic programming can find the optimal solution.\r\n A classic example is the "0-1 Knapsack Problem".\r\n\r\n Problem Statement:\r\n We are given a set of items, each with a weight and a value, and a knapsack that can carry a maximum weight.\r\n We must choose which items to put into the knapsack such that the total weight is less than or equal to the maximum weight and the total value is as large as possible.\r\n\r\n Important: We cannot take a fraction of an item (either take the whole 

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [18]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = distillation_formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 2,
        warmup_steps = 20,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 300,
        learning_rate = 2e-4,
        logging_steps = 10, # 너무 자주 로깅하면 오버헤드 발생
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = OUTPUT_DIR, # 구글 드라이브 경로로 설정
        report_to = "wandb", # Use this for WandB etc

        # --- 저장 관련 설정 추가 ---
        save_strategy = "steps",      # 스텝 기준으로 저장
        save_steps = 100,             # 100 스텝마다 저장
        save_total_limit = 3,         # 최대 3개의 체크포인트만 유지
        # --------------------------
    ),
)

In [19]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
19.434 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 90,990 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 2 x 1) = 64
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Step,Training Loss
10,1.073900
20,1.060600
30,1.119900
40,1.106900
50,1.092100


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
from transformers import TextStreamer

# 추론 모드로 전환
FastLanguageModel.for_inference(model)

# 테스트할 질문
question = "I have a list of numbers: 1, 2, 3, 4, 5. What is the sum of these numbers?"

# Llama-3.1 채팅 템플릿 적용
messages = [
    {"role": "user", "content": question},
    {"role": "assistant", "content": ""},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# 답변 생성 (스트리밍 방식)
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
print("="*30)
print("간단한 추론 테스트 결과:")
_ = model.generate(inputs, streamer=text_streamer, max_new_tokens=256, use_cache=True)
print("="*30)

In [ ]:
# LoRA 어댑터를 원본 모델에 병합하여 구글 드라이브에 저장합니다.
# 평가 프레임워크에서 사용하기 위해 이 과정이 권장됩니다.
print(f"\n모델을 {MERGED_MODEL_PATH} 경로에 병합하여 저장합니다...")
model.save_pretrained_merged(MERGED_MODEL_PATH, tokenizer, save_method="merged_16bit")
print("모델 병합 및 저장 완료!")

# ------------------------------------------------------------------------------
# AIME 벤치마크 평가 (AIME Benchmark with LightEval)
# ------------------------------------------------------------------------------

In [ ]:
%%capture

# LightEval 평가 패키지 설치
!pip install lighteval

In [ ]:
# LightEval 실행
# --model_args의 pretrained 경로가 위에서 병합한 모델의 경로와 일치해야 합니다.
# 처음에는 limit=50 정도로 테스트하고, 최종 평가는 limit 없이 실행하세요.
!lighteval \
    --model_args pretrained={MERGED_MODEL_PATH},trust_remote_code=True \
    --tasks aime:limit=50 \
    --override_bs 1

In [ ]:
# LightEval 실행
# --model_args의 pretrained 경로가 위에서 병합한 모델의 경로와 일치해야 합니다.
# 처음에는 limit=50 정도로 테스트하고, 최종 평가는 limit 없이 실행하세요.
!lighteval \
    --model_args pretrained={MERGED_MODEL_PATH},trust_remote_code=True \
    --tasks aime \
    --override_bs 1